# import timesheets

In [45]:
import os
import json
import hashlib
import traceback
from dateutil import parser
from datetime import datetime
from util.google_sheet import GGSheet

In [2]:
from util.frappyclient import FrappeClient

In [3]:
%env ACERP_ADMINISTRATOR=Administrator
%env ACERP_ADMINISTRATOR_PASS=Pandion123456@

env: ACERP_ADMINISTRATOR=Administrator
env: ACERP_ADMINISTRATOR_PASS=Pandion123456@


In [4]:
conn = FrappeClient("https://acerp.pandion.vn")
conn.login(os.environ['ACERP_ADMINISTRATOR'], os.environ['ACERP_ADMINISTRATOR_PASS'])

{'message': 'Logged In', 'home_page': '/app', 'full_name': 'Administrator'}

In [5]:
from util.util import mapping_row_with_attr, mapping_row_with_time_log, is_row_empty

In [6]:
from util.ts import Ts

In [7]:
ts = Ts(conn)

In [8]:
row_of_date = 3
row_start   = 5
row_end     = 1200

In [19]:
# import_key for task: task_id||
def handle_timesheet_row_with_parent_task(idx, row_data, row_attrs, row_date_header, company):

    # print(f"  Handle parent task at row: {idx}\n"
    #       f"  import_key: {row_attrs['import_key']}\n"
    #       f"  project_code: {row_attrs['project_code']}\n"
    #       f"  phase_name: {row_attrs['phase_name']}\n"
    #       f"  subject: {row_attrs['subject']}\n"
    #       f"  task: {row_attrs['task']}")
    task_doc = ts.get_task_doc_by_fields({
        "subject": row_attrs['subject'], 
        "project_code": row_attrs['project_code'], 
        "phase_name": row_attrs['phase_name'], 
        "is_group": 1})

    if task_doc:
        return task_doc
    else:
        # create new
        payload = {
            "subject": row_attrs["subject"],
            "project_code": row_attrs["project_code"],
            "phase_name": row_attrs["phase_name"],
            "is_group": 1,
        }
        print(f"{payload}")
        return ts.create_task(payload)

In [58]:
# import_key for task: timesheet_id||
def handle_timesheet_row_as_task(idx, row_data, row_attrs, row_date_header, 
                                 parent_task_row, parent_task_doc, company):
    time_logs_date, time_logsdate_string = mapping_row_with_time_log(row_data, row_date_header)
    
    # print(f"  Handle task at row: {idx}\n"
    #       f"  import_key: {row_attrs['import_key']}\n"
    #       f"  project_code: {row_attrs['project_code']}\n"
    #       f"  phase_name: {row_attrs['phase_name']}\n"
    #       # f"  subject: {row_attrs['subject']}\n"
    #       f"  employee_name: {row_attrs['employee_name']}\n"
    #       f"  task: {row_attrs['task']}\n"
    #       f"  parent task row: {parent_task_row}\n"
    #       f"  parent task doc: {parent_task_doc}\n"
    #       f"  time_logs: {time_logs_date}")

    ts_name = ts.get_timesheet_name_by_fields({
        "employee_code": row_attrs['employee_code'],
        "project_code": row_attrs['project_code'],
        "task": row_attrs['task'],
        "phase_name": row_attrs['phase_name']})

    if ts_name:
        print(f"Timesheet doc_name existed {ts_name}")
        
        # existed
        payload = {}
        payload["doctype"] = "Timesheet"
        payload["name"] = ts_name["name"]
        payload["employee"] = row_attrs["employee_code"]
        payload["customer"] = company
        payload["parent_project"] = row_attrs["project_code"]
        payload["custom_task"] = row_attrs["task"]
        payload["custom_phase"] = row_attrs["phase_name"]
        # payload["start_date"] = row_attrs["phase_name"]
        time_logs = []
        for date, hrs in time_logs_date.items():
            # check task here for update or new
            task_doc = ts.get_task_doc_by_fields({"subject": row_attrs['subject'], 
                                                  "project_code": row_attrs['project_code'], 
                                                  "phase_name": row_attrs['phase_name'], 
                                                  "is_group": 0, 
                                                  "parent_task": parent_task_doc['name']})
            
            if task_doc is None:
                print("task is not exists")
                # new task for timelogs
                task_payload = {
                    "subject": row_attrs["subject"],
                    "project_code": row_attrs["project_code"],
                    "phase_name": row_attrs["phase_name"],
                    "is_group": 0,
                    "parent_task": parent_task_doc['name']
                }
                task_doc = ts.create_task(task_payload)
                # print(f"new task doc: {task_doc}")

            if task_doc is None:
                print("fixme")
                continue

            hrs_ = 0.0
            try:
                hrs_ = float(hrs)
            except ValueError:
                hrs_ = 0.0
                
            time_logs.append({
                    "activity_type": row_attrs['activity_code'],
                    "from_time": date.strftime("%Y-%m-%d"),
                    "hours": hrs_,
                    "project": row_attrs['project_code'],
                    "task": task_doc['name'],
                    # "completed": excel_task_status == "Done",
                })
        payload["time_logs"] = time_logs
        # payload["is_group"] = 1
        print(f"{payload}")
        conn.update(payload)
        # pass
    else:
        # create new
        print(f"Timesheet need create new")
        payload = {}
        payload["doctype"] = "Timesheet"
        payload["employee"] = row_attrs["employee_code"]
        payload["parent_project"] = row_attrs["project_code"]
        payload["customer"] = company
        payload["custom_task"] = row_attrs["task"]
        payload["custom_phase"] = row_attrs["phase_name"]
        # payload["start_date"] = row_attrs["phase_name"]

        time_logs = []
        for date, hrs in time_logs_date.items():
            # check task here for update or new
            task_doc = ts.get_task_doc_by_fields({
                                "subject": row_attrs['subject'], 
                                "project_code": row_attrs['project_code'], 
                                "phase_name": row_attrs['phase_name'], 
                                "is_group": 0, 
                                "parent_task": parent_task_doc['name']})
            if task_doc is None:
                print("task is not exists")
                # new task for timelogs
                task_payload = {
                    "subject": row_attrs["subject"],
                    "project_code": row_attrs["project_code"],
                    "phase_name": row_attrs["phase_name"],
                    "is_group": 0,
                    "parent_task": parent_task_doc['name']
                }
                task_doc = ts.create_task(task_payload)
                # print(f"new task doc: {task_doc}")

            if task_doc is None:
                print("fixme")
                continue

            hrs_ = 0.0
            try:
                hrs_ = float(hrs)
            except ValueError:
                hrs_ = 0.0
            
            time_logs.append({
                    "activity_type": row_attrs['activity_code'],
                    "from_time": date.strftime("%Y-%m-%d"),
                    "hours": hrs_,
                    "project": row_attrs['project_code'],
                    "task": task_doc['name'],
                    # "completed": excel_task_status == "Done",
                })
        payload["time_logs"] = time_logs
        # payload["is_group"] = 1
        print(f"{payload}")
        ts.create_timesheet(payload)

    return True

In [11]:
# if row is empty
    # skip this row
# if col_B == "P"
    # handle as parent task
    # if col_A is not empty (parent task is created)
        # handle update parent task if any changes (compare hash)
    # else
        # handle as new parent task
# else
    # handle as task with timesheet
    # if col_A is not empty (timesheet is created)
        # handle update timesheet if any changes (compare hash)
    # else
        # handle as new timesheet

In [12]:
def handle_timesheet_row(idx, row_data, row_date_header, company):
    global current_parent_task_row
    global current_parent_task_doc
    row_attrs = mapping_row_with_attr(row_data)
    # NO project code
    if is_row_empty(row_attrs):
        return False

    # project code is not found ?
    print(f"Project code: {row_attrs['project_code']}")
    project_doc = ts.get_project_doc(row_attrs['project_code'])
    if project_doc is None:
        print(f"Project is not found: {row_attrs['project_code']}")
        return False
    
    if row_attrs['row_type'] == "P":
        ret = handle_timesheet_row_with_parent_task(idx, row_data, row_attrs, row_date_header, company)
        current_parent_task_row = idx
        current_parent_task_doc = ret
        return True
    else:
        # Handle with employee_name column: employee name is not found?
        print(f"Employee name: {row_attrs['employee_name']}")
        employee_doc = ts.get_employee_doc_by_name(row_attrs['employee_name'])
        if employee_doc is None:
            print(f"Employee is not found: {row_attrs['employee_name']}")
            return False
        else:
            print(f"Employee code: {employee_doc['name']}")
            row_attrs['employee_code'] = employee_doc['name']

        # handle as task with timesheet
        ret = handle_timesheet_row_as_task(idx, row_data, row_attrs, row_date_header, 
                                            current_parent_task_row, current_parent_task_doc, company)
        return ret
        

In [46]:
def handle_timesheet_sheet():
    try:
        # row_date_header = await get_row_date_header()
        # all_records = await ggSheet.get_all_row_values()
        
        current_parent_task_row = 0
        for idx, row_data in enumerate(all_records):
            if idx >= row_start and idx <= row_end:
                print(f"idx: {idx}")
                # print(row_data)
                handle_timesheet_row(idx, row_data, row_date_header, company)
                # break
    except Exception as err:
        print(f"Handle timesheet {worksheet_name} failed with: {err}")
        print(traceback.format_exc())

### BAP

In [ ]:
url_file="https://docs.google.com/spreadsheets/d/11y58PE9RM16JozFR8yT-oV3K4etIsp2oFmiCsYVaydk/edit#gid=1885142845"
# copied
# url_file="https://docs.google.com/spreadsheets/d/12bDZbCnJVkFTJTeMBmgsrAEejVKQCXa11M951ylOQVw/edit#gid=1885142845"
worksheet_name="BAP"

In [ ]:
ggSheet = GGSheet(url_file, worksheet_name)
row_date_header = await ggSheet.get_row_date_header(row_of_date)
all_records = await ggSheet.get_all_row_values()

In [20]:
company     = "BAP"
handle_timesheet_sheet()

idx: 5
Project code: HR01_ACO.00
idx: 6
Project code: HR01_ACO.00
Employee name: Nhật Phạm Hoàng
employee: {'name': 'HR-EMP-00007'}
Employee code: HR-EMP-00007
Timesheet doc_name existed {'name': 'TS-23-05-24-05061_HR-EMP-00007'}
{'doctype': 'Timesheet', 'name': 'TS-23-05-24-05061_HR-EMP-00007', 'employee': 'HR-EMP-00007', 'customer': 'BAP', 'parent_project': 'HR01_ACO.00', 'custom_task': 'Nghỉ lễ', 'custom_phase': 'Other', 'time_logs': [{'activity_type': '2000', 'from_time': '2024-04-27', 'hours': 8.0, 'project': 'HR01_ACO.00', 'task': '7iks43j1g1'}, {'activity_type': '2000', 'from_time': '2024-04-29', 'hours': 8.0, 'project': 'HR01_ACO.00', 'task': '7iks43j1g1'}, {'activity_type': '2000', 'from_time': '2024-04-30', 'hours': 8.0, 'project': 'HR01_ACO.00', 'task': '7iks43j1g1'}, {'activity_type': '2000', 'from_time': '2024-05-01', 'hours': 8.0, 'project': 'HR01_ACO.00', 'task': '7iks43j1g1'}]}
idx: 7
Project code: HR01_ACO.00
Employee name: Huyền Bùi Thị Thanh
employee: {'name': 'HR-EM

### TEAM1

In [69]:
url_file = "https://docs.google.com/spreadsheets/d/1M9T5wt7SIF6-nytkgDEK-LoSGfFgWxlExAy1qo6EPhU/edit#gid=1885142845"
worksheet_name = "T1"
company = "ACO"

In [70]:
await ggSheet.set_worksheet(url_file, worksheet_name)

In [71]:
row_date_header = await ggSheet.get_row_date_header(row_of_date)
all_records = await ggSheet.get_all_row_values()

In [73]:
handle_timesheet_sheet()

idx: 5
Project code: P16217_ANV.00
idx: 6
Project code: P16217_ANV.00
Employee name: Hưng Trần Ngọc
employee: {'name': 'HR-EMP-00042'}
Employee code: HR-EMP-00042
Timesheet doc_name existed {'name': 'TS-24-05-24-06773_HR-EMP-00042'}
{'doctype': 'Timesheet', 'name': 'TS-24-05-24-06773_HR-EMP-00042', 'employee': 'HR-EMP-00042', 'customer': 'ACO', 'parent_project': 'P16217_ANV.00', 'custom_task': 'Họp dự án', 'custom_phase': 'Construction', 'time_logs': [{'activity_type': '2001', 'from_time': '2024-04-04', 'hours': 4.0, 'project': 'P16217_ANV.00', 'task': '4fq74t753t'}, {'activity_type': '2001', 'from_time': '2024-04-11', 'hours': 4.0, 'project': 'P16217_ANV.00', 'task': '4fq74t753t'}]}
idx: 7
Project code: P16217_ANV.00
Employee name: Anh Trần Phan Thái
employee: {'name': 'HR-EMP-00084'}
Employee code: HR-EMP-00084
Timesheet doc_name existed {'name': 'TS-24-05-24-06774_HR-EMP-00084'}
{'doctype': 'Timesheet', 'name': 'TS-24-05-24-06774_HR-EMP-00084', 'employee': 'HR-EMP-00084', 'customer'

### TEAM_2

In [21]:
url_file="https://docs.google.com/spreadsheets/d/1FFkyht1r6aWqgh4i7EX6AkZRxCuSWen8qs75DaTCH_4/edit#gid=996503932"
# copied
# url_file = "https://docs.google.com/spreadsheets/d/1rgen0jYUoAzHRelVi6c2ATJs3IzYPD20wMcT63inKoI/edit#gid=996503932"
worksheet_name="T2"
company="ACO"

In [22]:
await ggSheet.set_worksheet(url_file, worksheet_name)

In [23]:
row_date_header = await ggSheet.get_row_date_header(row_of_date)
all_records = await ggSheet.get_all_row_values()

In [24]:
handle_timesheet_sheet()

idx: 5
Project code: P33422_HTP.00
idx: 6
Project code: P33422_HTP.00
Employee name: Khanh Dương Thái Như
employee: {'name': 'HR-EMP-00015'}
Employee code: HR-EMP-00015
Timesheet need create new
{'doctype': 'Timesheet', 'employee': 'HR-EMP-00015', 'parent_project': 'P33422_HTP.00', 'customer': 'ACO', 'custom_task': 'Thuyết minh TKTC', 'custom_phase': 'Construction', 'time_logs': []}
idx: 7
Project code: P33422_HTP.00
Employee name: Huy Phan Trịnh Hoàng
employee: {'name': 'HR-EMP-00069'}
Employee code: HR-EMP-00069
Timesheet doc_name existed {'name': 'TS-23-05-24-05227_HR-EMP-00069'}
{'doctype': 'Timesheet', 'name': 'TS-23-05-24-05227_HR-EMP-00069', 'employee': 'HR-EMP-00069', 'customer': 'ACO', 'parent_project': 'P33422_HTP.00', 'custom_task': 'Thuyết minh TKTC', 'custom_phase': 'Construction', 'time_logs': []}
idx: 8
Project code: P33422_HTP.00
Employee name: Bình Trịnh Thanh
employee: {'name': 'HR-EMP-00020'}
Employee code: HR-EMP-00020
Timesheet doc_name existed {'name': 'TS-23-05-2

### TEAM3

In [25]:
# team_3
url_file="https://docs.google.com/spreadsheets/d/1hO5er83L8qU6UITbsjacZg_A6SPW8MuFRBEfwQd1dys/edit#gid=370020233"
# copied
# url_file = "https://docs.google.com/spreadsheets/d/1LLKxyhIAbjMl8MXREZBGyH7r8puEtQATc5AiQLOIbh8/edit#gid=370020233"
worksheet_name="T3"
company="ACO"

In [26]:
await ggSheet.set_worksheet(url_file, worksheet_name)

In [27]:
row_date_header = await ggSheet.get_row_date_header(row_of_date)
all_records = await ggSheet.get_all_row_values()

In [28]:
handle_timesheet_sheet()

idx: 5
Project code: P000_ACO.00
idx: 6
Project code: P000_ACO.00
Employee name: Trung Đỗ Thành
employee: {'name': 'HR-EMP-00041'}
Employee code: HR-EMP-00041
Timesheet doc_name existed {'name': 'TS-23-05-24-05592_HR-EMP-00041'}
{'doctype': 'Timesheet', 'name': 'TS-23-05-24-05592_HR-EMP-00041', 'employee': 'HR-EMP-00041', 'customer': 'ACO', 'parent_project': 'P000_ACO.00', 'custom_task': 'Họp nội bộ', 'custom_phase': 'Other', 'time_logs': [{'activity_type': '2000', 'from_time': '2024-05-06', 'hours': 2.0, 'project': 'P000_ACO.00', 'task': 'd2946kmc0l'}]}
idx: 7
idx: 8
idx: 9
idx: 10
idx: 11
idx: 12
idx: 13
idx: 14
idx: 15
Project code: HR01_ACO.00
idx: 16
Project code: HR01_ACO.00
Employee name: Trung Đỗ Thành
employee: {'name': 'HR-EMP-00041'}
Employee code: HR-EMP-00041
Timesheet doc_name existed {'name': 'TS-23-05-24-05593_HR-EMP-00041'}
{'doctype': 'Timesheet', 'name': 'TS-23-05-24-05593_HR-EMP-00041', 'employee': 'HR-EMP-00041', 'customer': 'ACO', 'parent_project': 'HR01_ACO.00', 

### TEAM 4

In [29]:
# team_4
url_file="https://docs.google.com/spreadsheets/d/1p1vffmF2H-rOSdrNjdqMAXYTFlf9HkBX7FJjk2l4OwA/edit#gid=1885142845"
# copied
# url_file="https://docs.google.com/spreadsheets/d/11TQi-pF13BGS2LkrOHzO1EK-zAiZPw_Vop61d69oMFo/edit#gid=1885142845"
worksheet_name="T4"
company="ACO"

In [30]:
await ggSheet.set_worksheet(url_file, worksheet_name)

In [31]:
row_date_header = await ggSheet.get_row_date_header(row_of_date)
all_records = await ggSheet.get_all_row_values()

In [32]:
handle_timesheet_sheet()

idx: 5
Project code: HR01_ACO.00
idx: 6
Project code: HR01_ACO.00
Employee name: An Nguyễn Bình
employee: {'name': 'HR-EMP-00003'}
Employee code: HR-EMP-00003
Timesheet doc_name existed {'name': 'TS-23-05-24-05801_HR-EMP-00003'}
{'doctype': 'Timesheet', 'name': 'TS-23-05-24-05801_HR-EMP-00003', 'employee': 'HR-EMP-00003', 'customer': 'ACO', 'parent_project': 'HR01_ACO.00', 'custom_task': 'Nghỉ lễ giỗ tổ', 'custom_phase': 'Construction', 'time_logs': [{'activity_type': '4001', 'from_time': '2024-04-18', 'hours': 8.0, 'project': 'HR01_ACO.00', 'task': 'd62mj6kmal'}]}
idx: 7
Project code: HR01_ACO.00
Employee name: Son Trần Thị Minh
employee: {'name': 'HR-EMP-00005'}
Employee code: HR-EMP-00005
Timesheet doc_name existed {'name': 'TS-23-05-24-05802_HR-EMP-00005'}
{'doctype': 'Timesheet', 'name': 'TS-23-05-24-05802_HR-EMP-00005', 'employee': 'HR-EMP-00005', 'customer': 'ACO', 'parent_project': 'HR01_ACO.00', 'custom_task': 'Nghỉ lễ giỗ tổ', 'custom_phase': 'Construction', 'time_logs': [{'a

### TEAM 5

In [33]:
# team_5
url_file="https://docs.google.com/spreadsheets/d/1vTy7Qout9WoeZa31lHLQUw6CZecjEJOKUQlDIAmk1vk/edit#gid=1742134070"
# copied
# url_file="https://docs.google.com/spreadsheets/d/125OUpshHBj577ppiOUO_n73ZtgKDBBqLpmR0qST-IZA/edit#gid=1742134070"
worksheet_name="T5"
company="ACO"

In [34]:
await ggSheet.set_worksheet(url_file, worksheet_name)

In [35]:
row_date_header = await ggSheet.get_row_date_header(row_of_date)
all_records = await ggSheet.get_all_row_values()

In [38]:
handle_timesheet_sheet()

idx: 5
Project code: P39423_KNCR.00
idx: 6
Project code: P39423_KNCR.00
Employee name: 
employee: {}
Employee is not found: 
idx: 7
Project code: P39423_KNCR.00
Employee name: Thành Hồ Đức Thiện
employee: {'name': 'HR-EMP-00008'}
Employee code: HR-EMP-00008
Timesheet doc_name existed {'name': 'TS-23-05-24-06036_HR-EMP-00008'}
{'doctype': 'Timesheet', 'name': 'TS-23-05-24-06036_HR-EMP-00008', 'employee': 'HR-EMP-00008', 'customer': 'ACO', 'parent_project': 'P39423_KNCR.00', 'custom_task': 'Cập nhật phản hồi TKCS', 'custom_phase': 'Other', 'time_logs': [{'activity_type': '2303', 'from_time': '2024-04-01', 'hours': 8.0, 'project': 'P39423_KNCR.00', 'task': 'dco8m4aj1i'}, {'activity_type': '2303', 'from_time': '2024-04-02', 'hours': 8.0, 'project': 'P39423_KNCR.00', 'task': 'dco8m4aj1i'}, {'activity_type': '2303', 'from_time': '2024-04-03', 'hours': 8.0, 'project': 'P39423_KNCR.00', 'task': 'dco8m4aj1i'}, {'activity_type': '2303', 'from_time': '2024-04-04', 'hours': 8.0, 'project': 'P39423

### TEAM 6

In [39]:
# team_6
url_file="https://docs.google.com/spreadsheets/d/14m-Sk89NxBeh9pnqrtYr-Vdi92INzIFF1uxO7lpTDuk/edit#gid=1175515436"
# copied
# url_file="https://docs.google.com/spreadsheets/d/1CgHfrscwLZlU7gjk1IBGARlxEEsFML9YMiuEsaA9FWE/edit#gid=994132924"
worksheet_name="T6"
company="ACO"

In [40]:
await ggSheet.set_worksheet(url_file, worksheet_name)
row_date_header = await ggSheet.get_row_date_header(row_of_date)
all_records = await ggSheet.get_all_row_values()

In [41]:
handle_timesheet_sheet()

idx: 5
Project code: HR01_ACO.00
idx: 6
Project code: HR01_ACO.00
Employee name: Thiện Lê Phú
employee: {'name': 'HR-EMP-00043'}
Employee code: HR-EMP-00043
Timesheet doc_name existed {'name': 'TS-23-05-24-06074_HR-EMP-00043'}
{'doctype': 'Timesheet', 'name': 'TS-23-05-24-06074_HR-EMP-00043', 'employee': 'HR-EMP-00043', 'customer': 'ACO', 'parent_project': 'HR01_ACO.00', 'custom_task': '', 'custom_phase': 'Other', 'time_logs': [{'activity_type': '4001', 'from_time': '2024-04-18', 'hours': 8.0, 'project': 'HR01_ACO.00', 'task': 'bia0dlhuhn'}, {'activity_type': '4001', 'from_time': '2024-04-27', 'hours': 4.0, 'project': 'HR01_ACO.00', 'task': 'bia0dlhuhn'}, {'activity_type': '4001', 'from_time': '2024-04-29', 'hours': 8.0, 'project': 'HR01_ACO.00', 'task': 'bia0dlhuhn'}, {'activity_type': '4001', 'from_time': '2024-04-30', 'hours': 8.0, 'project': 'HR01_ACO.00', 'task': 'bia0dlhuhn'}, {'activity_type': '4001', 'from_time': '2024-05-01', 'hours': 8.0, 'project': 'HR01_ACO.00', 'task': 'bi

### TEAM CIVIL

In [63]:
# team_civil
url_file = "https://docs.google.com/spreadsheets/d/1qiVQptv2NK08Nr44ZapI8QTEHoDf1DhSpN0jkEe9_Ks/edit#gid=26030579"
# url_file="https://docs.google.com/spreadsheets/d/1qiVQptv2NK08Nr44ZapI8QTEHoDf1DhSpN0jkEe9_Ks/edit#gid=903792263"
# copied
# url_file=""
worksheet_name="CIVIL"
company="ACO"

In [64]:
await ggSheet.set_worksheet(url_file, worksheet_name)
row_date_header = await ggSheet.get_row_date_header(row_of_date)
all_records = await ggSheet.get_all_row_values()

In [65]:
handle_timesheet_sheet()

idx: 5
idx: 6
Project code: P33422_HTP.00
idx: 7
Project code: P33422_HTP.00
Employee name: 
employee: {}
Employee is not found: 
idx: 8
Project code: P33422_HTP.00
Employee name: Minh Nguyễn Thành
employee: {'name': 'HR-EMP-00078'}
Employee code: HR-EMP-00078
Timesheet doc_name existed {'name': 'TS-24-05-24-06384_HR-EMP-00078'}
{'doctype': 'Timesheet', 'name': 'TS-24-05-24-06384_HR-EMP-00078', 'employee': 'HR-EMP-00078', 'customer': 'ACO', 'parent_project': 'P33422_HTP.00', 'custom_task': 'MB tổ chức giao thông & chi tiết', 'custom_phase': 'Other', 'time_logs': []}
idx: 9
Project code: P33422_HTP.00
Employee name: Minh Nguyễn Thành
employee: {'name': 'HR-EMP-00078'}
Employee code: HR-EMP-00078
Timesheet doc_name existed {'name': 'TS-24-05-24-06385_HR-EMP-00078'}
{'doctype': 'Timesheet', 'name': 'TS-24-05-24-06385_HR-EMP-00078', 'employee': 'HR-EMP-00078', 'customer': 'ACO', 'parent_project': 'P33422_HTP.00', 'custom_task': 'Đấu nối đường Ven Biển hiện trạng', 'custom_phase': 'Other', 

### TEAM RND

In [66]:
# team_rnd
url_file="https://docs.google.com/spreadsheets/d/15JPQLVqA_WDLmK00AsCnYrv1xqGzWwV9xbung_JDBmU/edit#gid=1885142845"
# copied
# url_file="https://docs.google.com/spreadsheets/d/1EhlTU0LKzv-OsngOaZ9rXpzOPXmAureMjHrRo2OecTw/edit#gid=1885142845"
worksheet_name="RnD"
company="ACO"

In [67]:
await ggSheet.set_worksheet(url_file, worksheet_name)
row_date_header = await ggSheet.get_row_date_header(row_of_date)
all_records = await ggSheet.get_all_row_values()

In [68]:
handle_timesheet_sheet()

idx: 5
Project code: P000_ACO.00
idx: 6
Project code: P000_ACO.00
Employee name: Việt Nguyễn Hồng
employee: {'name': 'HR-EMP-00040'}
Employee code: HR-EMP-00040
Timesheet doc_name existed {'name': 'TS-23-05-24-06162_HR-EMP-00040'}
{'doctype': 'Timesheet', 'name': 'TS-23-05-24-06162_HR-EMP-00040', 'employee': 'HR-EMP-00040', 'customer': 'ACO', 'parent_project': 'P000_ACO.00', 'custom_task': 'Gernal Task', 'custom_phase': 'Other', 'time_logs': [{'activity_type': '2000', 'from_time': '2024-05-20', 'hours': 2.0, 'project': 'P000_ACO.00', 'task': 'diit0en9af'}]}
idx: 7
idx: 8
idx: 9
idx: 10
idx: 11
idx: 12
Project code: I00124_RSCI.00
idx: 13
Project code: I00124_RSCI.00
Employee name: Sokkheang Chan
employee: {'name': 'HR-EMP-00050'}
Employee code: HR-EMP-00050
Timesheet doc_name existed {'name': 'TS-23-05-24-06163_HR-EMP-00050'}
{'doctype': 'Timesheet', 'name': 'TS-23-05-24-06163_HR-EMP-00050', 'employee': 'HR-EMP-00050', 'customer': 'ACO', 'parent_project': 'I00124_RSCI.00', 'custom_task